In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import albumentations as A
import numpy as np
import segmentation_models_pytorch as smp
import torch
from scripts.training import get_best_available_device, load_updated_params
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
import cv2 as cv
from scripts.utils import SegmentationDataset, split_data, get_prediction, filter_circles

In [34]:
test_directory = "data/test"

image_paths, _, _, _ = split_data(test_directory, 0.0, 'inference')

In [35]:
seg_tf = A.Compose([
    A.Resize(width=600, height=400, always_apply=True),
    A.PadIfNeeded(min_height=416, min_width=608, always_apply=True),
])

In [36]:
val_ds = SegmentationDataset(
    image_paths=image_paths,
    transform=seg_tf,
)

loader = DataLoader(val_ds, batch_size=1)

In [38]:
model = smp.Unet(
    encoder_name="resnet50",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
)

model = load_updated_params(model, 'models/segmentation_20eps.pt')
model = model.to(get_best_available_device())

In [47]:
for image, _, filename in loader:
    
    predicted = get_prediction(model, image)
    
    circles = cv.HoughCircles((predicted * 255).astype(np.uint8), cv.HOUGH_GRADIENT, 1, 20, param1=200, param2=10, minRadius=15, maxRadius=55)
    circles = filter_circles(circles)
    
    kala = 1

KeyboardInterrupt: 